In [1]:
from libraries import *
from functions import *
from fitting import *
from util import *
%matplotlib qt
data[0]

/home/alanyu/anaconda3/lib/python3.8/site-packages/pysis/env.py:33: RuntimeWarning: Could not find isis. Is `ISISROOT` set?
  warnings.warn('Could not find isis. Is `ISISROOT` set?', RuntimeWarning)


Current progress: 50000
Current progress: 100000
Current progress: 150000
Current progress: 200000
Current progress: 250000
DONE


NameError: name 'reasonable_output' is not defined

### Plot result from pickles

Given an index and a channel, plot the spectra of the channel along with the fitted functions

In [3]:
def plot_result(entry, channel):
    entry = data[entry]
    #print(entry['regular_fit']['powerlaw'])
    fig, axs = plt.subplots(2, 2)
    #print('powerlaw' in entry)
    if 'powerlaw' in entry['regular_fit']:
        axs[0,0].plot(vims_wave, entry['spectrum']-powerlaw_no_constant(vims_wave, *entry['regular_fit']['powerlaw'].values()), label = "Spectra (powerlaw adjusted)")
        axs[0,0].plot(vims_wave, gaussian(vims_wave, *entry['regular_fit'][channel]['gaussian'].values()), label = 'Gaussian')
        axs[0,0].plot(vims_wave, skew(vims_wave, *(entry['regular_fit'][channel]['skew'][x] for x in entry['regular_fit'][channel]['skew'] if 'peak' not in x)), label = 'Skew')
        
    else:
        axs[0,0].plot(vims_wave, entry['spectrum'], label = "Spectrum (no powerlaw adjustment)")
        axs[0,0].plot(vims_wave, gaussian(vims_wave, *entry['regular_fit'][channel]['gaussian'].values()), label = 'Gaussian')
        axs[0,0].plot(vims_wave, skew(vims_wave, *(entry['regular_fit'][channel]['skew'][x] for x in entry['regular_fit'][channel]['skew'] if 'peak' not in x)), label = 'Skew')
    
    axs[0,1].plot(vims_wave, entry['stefan_spectrum'], label = "Stefan")
    axs[0,1].plot(vims_wave, gaussian(vims_wave, *entry['stefan_fit'][channel]['gaussian'].values()), label = "Gaussian")
    axs[0,1].plot(vims_wave, skew(vims_wave, *(entry['stefan_fit'][channel]['skew'][x] for x in entry['stefan_fit'][channel]['skew'] if 'peak' not in x)), label = "Skew")
    
    if 'powerlaw' in entry['stefan-plaw_fit']:
        axs[1,0].plot(vims_wave, entry['stefan_spectrum']-powerlaw_no_constant(vims_wave, *entry['stefan-plaw_fit']['powerlaw'].values()), label = "Stefan spectra (powerlaw adjusted)")
        axs[1,0].plot(vims_wave, gaussian(vims_wave, *entry['stefan-plaw_fit'][channel]['gaussian'].values()), label = 'Gaussian')
        axs[1,0].plot(vims_wave, skew(vims_wave, *(entry['stefan-plaw_fit'][channel]['skew'][x] for x in entry['stefan-plaw_fit'][channel]['skew'] if 'peak' not in x)), label = 'Skew')
        
    else:
        axs[1,0].plot(vims_wave, entry['stefan_spectrum'], label = "Stefan spectrum (no powerlaw adjustment)")
        axs[1,0].plot(vims_wave, gaussian(vims_wave, *entry['stefan-plaw_fit'][channel]['gaussian'].values()), label = 'Gaussian')
        axs[1,0].plot(vims_wave, skew(vims_wave, *(entry['stefan-plaw_fit'][channel]['skew'][x] for x in entry['stefan-plaw_fit'][channel]['skew'] if 'peak' not in x)), label = 'Skew')
    
    
    axs[0,0].legend()
    axs[0,1].legend()
    axs[1,0].legend()
    
    plt.show()

plot_result(0,'2.0')

In [3]:
filtered_data = filter_data(data)
len(filtered_data)

{'name': 'V1753535313_1.ir_rc20final.cub', 'sequenceid': 'S79', 'observationid': 'VIMS_195TI_HIRES003_PRIME', 'latitude': -14.192410526012, 'longitude': 113.08447714133, 'phase': 79.247331641247, 'incidence': 68.628679969023, 'emission': 26.016340587574, 'distance': 7368.7025357152, 'lineres': 1965.8545564098, 'sampres': 2466.4575678517, 'scposx': -2735.0425878636, 'scposy': 5677.0692141084, 'scposz': -3819.4507773567, 'sunposx': -1380094499.4225, 'sunposy': 147413363.40772, 'sunposz': 488789535.78307, 'ettime': 428112953.54426, 'starttime': '2013-07-26 12:12:41.994', 'exposureduration': 160.0, 'spectrum': array([ 0.0404514 ,  0.0663658 ,  0.086725  ,  0.0854239 ,  0.0644469 ,
        0.0462576 ,  0.0390803 ,  0.0374057 ,  0.048135  ,  0.0629884 ,
        0.0766593 ,  0.0830114 ,  0.0691124 ,  0.0468681 ,  0.0240458 ,
        0.0202946 ,  0.0128574 ,  0.016177  ,  0.021298  ,  0.032576  ,
        0.0397113 ,  0.0422687 ,  0.0508583 ,  0.059501  ,  0.0663442 ,
        0.042854  ,  0.026

Total 89372 entries removed


190420

### Time Bucketing and Restricting Viewing Geometry

Call time_bucket to bucket all entries into k (default 20) equally spaced time buckets. Can also use the function viewing_geometry to specify the range of incidence, emission, and phase angles. 

In [4]:
buckets = time_bucket(filtered_data)
[(x, len(buckets[x])) for x in buckets]

[((152047486.19852, 170097356.313667), 3465),
 ((170097356.313667, 188147226.428814), 1211),
 ((188147226.428814, 206197096.543961), 3524),
 ((206197096.543961, 224246966.65910798), 11486),
 ((224246966.65910798, 242296836.77425498), 40716),
 ((242296836.77425498, 260346706.88940197), 11958),
 ((260346706.88940197, 278396577.00454897), 1480),
 ((278396577.00454897, 296446447.11969596), 3327),
 ((296446447.11969596, 314496317.23484296), 10047),
 ((314496317.23484296, 332546187.34998995), 31004),
 ((332546187.34998995, 350596057.46513695), 8044),
 ((350596057.46513695, 368645927.58028394), 6366),
 ((368645927.58028394, 386695797.69543093), 14053),
 ((386695797.69543093, 404745667.8105779), 6333),
 ((404745667.8105779, 422795537.9257249), 14100),
 ((422795537.9257249, 440845408.0408719), 18262),
 ((440845408.0408719, 458895278.1560189), 406),
 ((458895278.1560189, 476945148.2711659), 1174),
 ((476945148.2711659, 494995018.3863129), 1972),
 ((494995018.3863129, 513044888.5014599), 1491)]

In [5]:
viewing_geometry

<function util.viewing_geometry(data, incidence_range=(-inf, inf), emission_range=(-inf, inf), phase_range=(-inf, inf))>

### Rainbow Plots

Using the data split into time buckets, plot one gaussian for each time bucket to achieve a rainbow-like effect.

![Two-micron rainbow](rainbow.png "Rainbow")

In [6]:
def plot_average_fits_rainbow(buckets, channel = '2.0', fit_type = 'regular_fit'):
    ncolors = len(buckets)
    colors = plt.cm.nipy_spectral(np.linspace(0.05,0.95,ncolors))
    
    parameters = {x: {} for x in buckets}
    
    for i, bucket in enumerate(buckets):
        mean = []
        amplitudes = []
        width = []
        
        for entry in buckets[bucket]:
            amplitudes.append(entry[fit_type][channel]['gaussian']['amp'])
            width.append(entry[fit_type][channel]['gaussian']['sigma'])
            mean.append(entry[fit_type][channel]['gaussian']['mean'])
        
        median = len(mean) // 2
        
        parameters[bucket]['mean'] = mean[median]
        parameters[bucket]['width'] = width[median]
        parameters[bucket]['amplitudes'] = amplitudes[median]
        
    for i, bucket in enumerate(parameters):
        if parameters[bucket].get('mean', None) != None:
            mean_, sigma_, amp_ = parameters[bucket]['mean'], parameters[bucket]['width'], parameters[bucket]['amplitudes']
            x = np.linspace(0, 5, 1000)
            plt.plot(x, gaussian(x, mean_, .1, 1), color = colors[i], label = bucket)

    plt.legend()

In [7]:
plot_average_fits_rainbow(buckets, channel = '2.0', fit_type = 'stefan_fit')

### Temporal Plots

Plot a multi-panel figure given a list of pixels displaying how best fit parameters change with time.

In [7]:
def plot_temporal(data, channel = '2.0', sample_every = 100):
    
    fig, axs = plt.subplots(4,3)
    
    info = {'stefan_g': {x: [] for x in data[0]['stefan_fit'][channel]['gaussian']},
            'stefan_s': {x: [] for x in data[0]['stefan_fit'][channel]['skew']},
            'regular_g': {x: [] for x in data[0]['regular_fit'][channel]['gaussian']},
            'regular_s': {x: [] for x in data[0]['regular_fit'][channel]['skew']},
            'ettimes': [],
            'starttimes': [],
            'jtimes': [],
            'incidences': [],
            'emissions': [],
            'phases': [],
            'powerlaw': {'scales': [], 'exps': []}}
            
    for i,entry in enumerate(data):
        if i%sample_every == 0:
            for x in entry['stefan_fit'][channel]['gaussian']:
                info['stefan_g'][x].append(entry['stefan_fit'][channel]['gaussian'][x])

            for x in entry['stefan_fit'][channel]['skew']:
                info['stefan_s'][x].append(entry['stefan_fit'][channel]['skew'][x])

            for x in entry['regular_fit'][channel]['gaussian']:
                info['regular_g'][x].append(entry['regular_fit'][channel]['gaussian'][x])

            for x in entry['regular_fit'][channel]['skew']:
                info['regular_s'][x].append(entry['regular_fit'][channel]['skew'][x]) 

            info['ettimes'].append(entry['ettime'])
            info['starttimes'].append(entry['starttime'][:-19])
            info['incidences'].append(entry['incidence'])
            info['emissions'].append(entry['emission'])
            info['phases'].append(entry['phase'])
            info['jtimes'].append(Time(entry['starttime'], format = 'iso').jd)

            if 'powerlaw' in entry['regular_fit']:
                info['powerlaw']['scales'].append(entry['regular_fit']['powerlaw']['scale'])
                info['powerlaw']['exps'].append(entry['regular_fit']['powerlaw']['exponent'])
            else:
                info['powerlaw']['scales'].append(None)
                info['powerlaw']['exps'].append(None)

    print(len(info['stefan_g']['mean']))
    axs[0, 0].xaxis.set_tick_params(rotation=30, labelsize=5)
    axs[0, 0].scatter(info['jtimes'], info['regular_g']['mean'], alpha = .3)
    axs[0, 0].set_xticks(info['jtimes'][::100])
    axs[0, 0].set_xticklabels(sorted(list(set(info['starttimes'][::100]))))
    axs[0, 0].set_title('Regular Gaussian Mean')
    p = np.polyfit(info['jtimes'], info['regular_g']['mean'], 1)
    fit = np.polyval(p, info['jtimes'])
    axs[0, 0].plot(info['jtimes'], fit, c = 'r')
    print("Regular Gaussian Mean", fit[0], fit[-1], p)
    
    axs[0, 1].scatter(info['jtimes'], info['regular_g']['sigma'], alpha = .3)
    axs[0, 1].set_title('Regular Gaussian Width')
    axs[0, 1].set_ylim([0,.2])
    
    axs[0, 2].scatter(info['jtimes'], info['regular_g']['amp'], alpha = .3)
    axs[0, 2].set_title('Regular Gaussian Amplitude')
    
    axs[1, 0].scatter(info['jtimes'], info['regular_s']['peak_channel'], alpha = .3)
    axs[1, 0].set_title('Regular Skew Peak Channel')
    p = np.polyfit(info['jtimes'], info['regular_s']['peak_channel'], 1)
    fit = np.polyval(p, info['jtimes'])
    axs[1, 0].plot(info['jtimes'], fit, c = 'r')
    print("Regular Skew Peak Channel", fit[0], fit[-1], p)
    
    axs[1, 1].scatter(info['jtimes'], info['regular_s']['peak_value'], alpha = .3)
    axs[1, 1].set_title('Regular Skew Peak Value')
    
    axs[1, 2].scatter(info['jtimes'], info['powerlaw']['scales'], alpha = .3)
    axs[1, 2].set_title('Powerlaw Scale')
    axs[1, 2].set_ylim(-.5, .2)
    
    axs[2, 0].scatter(info['jtimes'], info['powerlaw']['exps'], alpha = .3)
    axs[2, 0].set_title('Powerlaw Exponent')
    axs[2, 0].set_ylim(-5,0)
    
    axs[2, 1].scatter(info['jtimes'], info['stefan_g']['mean'], alpha = .3)
    axs[2, 1].set_title('Stefan Gaussian Mean')
    p = np.polyfit(info['jtimes'], info['stefan_g']['mean'], 1)
    fit = np.polyval(p, info['jtimes'])
    axs[2, 1].plot(info['jtimes'], fit, c = 'r')
    print("Stefan Gaussian Mean", fit[0], fit[-1], p)
    
    axs[2, 2].scatter(info['jtimes'], info['stefan_g']['sigma'], alpha = .3)
    axs[2, 2].set_title('Stefan Gaussian Width')
    axs[2, 2].set_ylim([0,0.2])
    
    axs[3, 0].scatter(info['jtimes'], info['stefan_g']['amp'], alpha = .3)
    axs[3, 0].set_title('Stefan Gaussian Amplitude')
    axs[3, 0].set_ylim([0,1])
    
    axs[3, 1].scatter(info['jtimes'], info['stefan_s']['peak_channel'], alpha = .3)
    axs[3, 1].set_title('Stefan Skew Peak Channel')
    p = np.polyfit(info['jtimes'], info['stefan_s']['peak_channel'], 1)
    fit = np.polyval(p, info['jtimes'])
    axs[3, 1].plot(info['jtimes'], fit, c = 'r')
    print("Stefan Skew Peak Channel", fit[0], fit[-1], p)
    
    axs[3, 2].scatter(info['jtimes'], info['stefan_s']['peak_value'], alpha = .3)
    axs[3, 2].set_title('Stefan Skew Peak Value')
    axs[3, 2].set_ylim([0, .25])
    plt.tight_layout(pad=0.00001)
    
plot_temporal(filtered_data)
    

1905
Regular Gaussian Mean 2.02436676039215 2.0206500882367067 [-3.83571510e-06  1.14421682e+01]
Regular Skew Peak Channel 2.0341426323866654 2.0338523561503736 [-2.99573623e-07  2.76968341e+00]
Stefan Gaussian Mean 2.0192379660641997 2.015394141271834 [-3.96694037e-06  1.17592357e+01]
Stefan Skew Peak Channel 2.029639813132249 2.0248481981650723 [-4.94508774e-06  1.41712752e+01]


### Slopes for all (incidence, emission) bins

Bin incidence and emission into bins of range 10 each and find linear fit for each bin.

In [8]:
min(data, key = lambda x: x['incidence'])['incidence'], max(data, key = lambda x: x['incidence'])['incidence']

(0.15913687184228, 89.998882744836)

In [9]:
min(data, key = lambda x: x['emission'])['emission'], max(data, key = lambda x: x['emission'])['emission']

(0.017992899122658, 85.654974175337)

In [13]:
def assign_bins(data, spread):
    bins = []
    for i in range(90//spread):
        current = [((spread*i),(spread*(i+1)))]
        for j in range(90//spread):
            bins.append(current + [(spread*j,spread*(j+1))])
    
    assignment = {tuple(x): [] for x in bins}
    for entry in data:
        for _bin in bins:
            if _bin[0][0] <= entry['incidence'] <= _bin[0][1] and _bin[1][0] <= entry['emission'] <= _bin[1][1]:
                assignment[tuple(_bin)].append(entry)
    return assignment

def analyze_bins(data, spread, channel, fit_type, function, mean_field):#:, weighted = False):
    
    binned_data = assign_bins(data, spread)
    
    bin_shifts = {b: None for b in binned_data}
    
    for bin in binned_data:
        if not binned_data[bin] or len(binned_data[bin]) < 1000:
            print(f"Skipping bin {bin} due to little/no values being present.")
            continue
        print(f"Bin {bin} has length {len(binned_data[bin])}")
        times = [Time(entry['starttime'], format = 'iso').jd for entry in binned_data[bin]]
        means = [entry[fit_type][channel][function][mean_field] for entry in binned_data[bin]]
        
        
        p = np.polyfit(times, means, 1)
        fit = np.polyval(p, times)
        
        bin_shifts[bin] = [fit[0], fit[-1], p]
    
    #if not weighted:
    print("MEAN", np.mean([x[-1][0] for x in bin_shifts.values() if x != None]))
    #print("MEDIAN", np.median([x[-1][0] for x in bin_shifts.values() if x != None]))
    #else:
    print("WEIGHTED MEAN", np.average([bin_shifts[x][-1][0] for x in bin_shifts if bin_shifts[x]!= None], weights = [len(binned_data[x]) for x in bin_shifts if bin_shifts[x] != None]))
    print("MEDIAN", np.median([x[-1][0] for x in bin_shifts.values() if x != None]))

    return bin_shifts

In [17]:
analyze_bins(filtered_data, 10, '1.0', 'stefan_fit', 'gaussian', 'mean')#, False)

Bin ((0, 10), (0, 10)) has length 1362
Bin ((0, 10), (10, 20)) has length 2015
Bin ((0, 10), (20, 30)) has length 3015
Bin ((0, 10), (30, 40)) has length 2788
Bin ((0, 10), (40, 50)) has length 1550
Skipping bin ((0, 10), (50, 60)) due to little/no values being present.
Skipping bin ((0, 10), (60, 70)) due to little/no values being present.
Skipping bin ((0, 10), (70, 80)) due to little/no values being present.
Skipping bin ((0, 10), (80, 90)) due to little/no values being present.
Bin ((10, 20), (0, 10)) has length 3963
Bin ((10, 20), (10, 20)) has length 5977
Bin ((10, 20), (20, 30)) has length 6656
Bin ((10, 20), (30, 40)) has length 7140
Bin ((10, 20), (40, 50)) has length 2876
Bin ((10, 20), (50, 60)) has length 1286
Skipping bin ((10, 20), (60, 70)) due to little/no values being present.
Skipping bin ((10, 20), (70, 80)) due to little/no values being present.
Skipping bin ((10, 20), (80, 90)) due to little/no values being present.
Bin ((20, 30), (0, 10)) has length 1118
Bin ((20,

{((0, 10), (0, 10)): [1.060133656402607,
  1.0601337687758194,
  array([ 9.85372184e-07, -1.35902317e+00])],
 ((0, 10), (10, 20)): [1.05944548128448,
  1.0626594123692215,
  array([-2.86833780e-06,  8.10254662e+00])],
 ((0, 10), (20, 30)): [1.0598731612753207,
  1.0621079047112651,
  array([-1.99440323e-06,  5.95705918e+00])],
 ((0, 10), (30, 40)): [1.060219521349595,
  1.060780936436994,
  array([-7.44717307e-07,  2.88864471e+00])],
 ((0, 10), (40, 50)): [1.0601273877332957,
  1.0608293489672587,
  array([-9.31225305e-07,  3.34646578e+00])],
 ((0, 10), (50, 60)): None,
 ((0, 10), (60, 70)): None,
 ((0, 10), (70, 80)): None,
 ((0, 10), (80, 90)): None,
 ((10, 20), (0, 10)): [1.0637398914347749,
  1.0632791659427046,
  array([-3.74611901e-06,  1.02578418e+01])],
 ((10, 20), (10, 20)): [1.0599409274888707,
  1.063997188939302,
  array([-3.62005981e-06,  9.94886871e+00])],
 ((10, 20), (20, 30)): [1.0604090013770646,
  1.059080767535626,
  array([-1.77690425e-06,  5.42308178e+00])],
 ((10,

```
results from previous: (mean, weighted mean, median)


analyze_bins(filtered_data, 90, '2.0', 'regular_fit', 'gaussian', 'mean') => -3.77e-06
analyze_bins(filtered_data, 90, '2.0', 'stefan_fit', 'gaussian', 'mean') => -3.93e-06

analyze_bins(filtered_data, 10, '2.0', 'stefan_fit', 'gaussian', 'mean') => -2.93e-06, -2.85e-06, -2.85e-06
analyze_bins(filtered_data, 10, '1.6', 'stefan_fit', 'gaussian', 'mean') => -3.47e-06, -3.10e-06, -2.99e-06
analyze_bins(filtered_data, 10, '1.2', 'stefan_fit', 'gaussian', 'mean') => -3.21e-06, -3.14e-06, -3.08e-06
analyze_bins(filtered_data, 10, '1.0', 'stefan_fit', 'gaussian', 'mean') => -2.42e-06, -2.41e-06, -2.39e-06

analyze_bins(filtered_data, 10, '2.0', 'regular_fit', 'gaussian', 'mean') => -2.97e-06, -2.98e-06, -2.96e-06
analyze_bins(filtered_data, 10, '1.6', 'regular_fit', 'gaussian', 'mean') => -3.53e-06, -3.22e-06, -3.14e-06
analyze_bins(filtered_data, 10, '1.2', 'regular_fit', 'gaussian', 'mean') => -2.49e-06, -2.69e-06, -2.70e-06
analyze_bins(filtered_data, 10, '1.0', 'regular_fit', 'gaussian', 'mean') => -2.46e-06, , -2.60e-06, -2.66e-06

skew weird because discrete values
```